In [2]:
from google.colab import files
uploaded = files.upload()

Saving bicycle.csv to bicycle.csv
Saving bicycle_dup.csv to bicycle_dup.csv
Saving bicycle_out.csv to bicycle_out.csv
Saving customer.xlsx to customer.xlsx
Saving sales.csv to sales.csv


In [65]:
import pandas as pd
import numpy as np
sales = pd.read_csv( './sales.csv', encoding='utf-8')
customer = pd.read_excel('./customer.xlsx')
print(sales.head())
print(customer.head())

      purchase_date item_name  item_price customer_name
0  2019-06-13 18:02       상품A       100.0           김가온
1  2019-07-13 13:05     상 품 S         NaN           김우찬
2  2019-05-11 19:42     상 품 a         NaN           김유찬
3  2019-02-12 23:40       상품Z      2600.0           김재현
4   2019-04-22 3:09       상품a         NaN           김강현
   고객이름  지역                  등록일
0  김 현성  H시  2018-01-04 00:00:00
1  김 도윤  E시                42782
2  김 지한  A시  2018-01-07 00:00:00
3  김 하윤  F시                42872
4  김 시온  E시                43127


In [66]:
#문제해결 05
#상품명 오류류
sales['item_name'] = sales['item_name'].str.replace(" ", "")
sales['item_name'] = sales["item_name"].str.upper()
pd.unique(sales['item_name'])

array(['상품A', '상품S', '상품Z', '상품V', '상품O', '상품U', '상품L', '상품C', '상품I',
       '상품R', '상품X', '상품G', '상품P', '상품Q', '상품Y', '상품N', '상품W', '상품E',
       '상품K', '상품B', '상품F', '상품D', '상품M', '상품H', '상품T', '상품J'],
      dtype=object)

In [67]:
#customer 날자 형식으로 바꾸기기
sales['month'] = pd.to_datetime(sales['purchase_date']).dt.strftime('%Y%M')
print(sales['month'])

0       201902
1       201905
2       201942
3       201940
4       201909
         ...  
2994    201956
2995    201903
2996    201914
2997    201956
2998    201931
Name: month, Length: 2999, dtype: object


In [68]:
#등록일 변경
date = customer['등록일'].astype('str').str.isdigit()
date.sum()
newDate1 = pd.to_timedelta(customer.loc[date,'등록일'].astype('float'),unit='D') + pd.to_datetime('1900/01/01') # -2
newDate1
newDate2 = pd.to_datetime( customer.loc[~date,'등록일'])
customer['등록일'] = pd.concat([newDate1, newDate2])
customer


,고객이름,지역,등록일
0,김 현성,H시,2018-01-04
1,김 도윤,E시,2017-02-18
2,김 지한,A시,2018-01-07
3,김 하윤,F시,2017-05-19
4,김 시온,E시,2018-01-29
...,...,...,...
195,김 재희,G시,2017-06-20
196,김 도영,E시,2018-06-20
197,김 이안,F시,2017-04-29
198,김 시현,H시,2019-04-19


In [70]:
#purchase_data의 오류가 있는지 월별로 판매개수 check
date = sales['purchase_date'].astype('str').str.isdigit()
newDate1 = pd.to_timedelta(sales.loc[date,'purchase_date'].astype('float'),unit='D') + pd.to_datetime('1900/01/01') # -2
newDate2 = pd.to_datetime( sales.loc[~date,'purchase_date'])
customer['purchase_date'] = pd.concat([newDate1, newDate2])
sales['month'] = pd.to_datetime(sales['purchase_date']).dt.strftime('%Y%m')
print(sales['month'])

0       201906
1       201907
2       201905
3       201902
4       201904
         ...  
2994    201902
2995    201906
2996    201903
2997    201907
2998    201907
Name: month, Length: 2999, dtype: object


In [74]:
#가격 결측치 수정
sales_null = sales['item_price'].isnull()

sales_null = sales['item_price'].isnull()

for i in list( sales.loc[ sales_null, 'item_name'].unique() ):
  price = sales.loc[ ( ~sales_null ) & (sales['item_name'] == i ), 'item_price'].max()
  sales['item_price'].loc[ (sales_null) & (sales['item_name'] == i )] = price
sales.isnull().sum()


<ipython-input-74-86eada539376>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales['item_price'].loc[ (sales_null) & (sales['item_name'] == i )] = price


purchase_date    0
item_name        0
item_price       0
customer_name    0
month            0
dtype: int64

In [77]:
customer['고객이름'] = customer['고객이름'].str.replace(" ", "")#object타입인 파일 공백 제거거
customer


,고객이름,지역,등록일,purchase_date
0,김현성,H시,2018-01-04,2019-06-13 18:02:00
1,김도윤,E시,2017-02-18,2019-07-13 13:05:00
2,김지한,A시,2018-01-07,2019-05-11 19:42:00
3,김하윤,F시,2017-05-19,2019-02-12 23:40:00
4,김시온,E시,2018-01-29,2019-04-22 03:09:00
...,...,...,...,...
195,김재희,G시,2017-06-20,2019-07-19 11:50:00
196,김도영,E시,2018-06-20,2019-07-17 02:53:00
197,김이안,F시,2017-04-29,2019-02-25 12:01:00
198,김시현,H시,2019-04-19,2019-04-13 13:05:00


In [78]:
#피봇으로 가능
customer['등록연월'] = customer['등록일'].dt.strftime('%Y%m')
res= customer.groupby('등록연월')['고객이름'].count()
res


등록연월
201701    15
201702    11
201703    14
201704    15
201705    13
201706    14
201707    17
201801    13
201802    15
201803    17
201804     5
201805    19
201806    13
201807    17
201904     2
Name: 고객이름, dtype: int64

In [84]:
#join
join = pd.merge(sales, customer, left_on='customer_name', right_on="고객이름", how="left")
join = join.drop('고객이름', axis=1)
join

,purchase_date_x,item_name,item_price,customer_name,month,지역,등록일,purchase_date_y,등록연월
0,2019-06-13 18:02,상품A,100.0,김가온,201906,C시,2017-01-26,2019-05-22 16:50:00,201701
1,2019-07-13 13:05,상품S,1900.0,김우찬,201907,C시,2018-04-07,2019-07-27 00:08:00,201804
2,2019-05-11 19:42,상품A,100.0,김유찬,201905,A시,2018-06-19,2019-06-29 17:42:00,201806
3,2019-02-12 23:40,상품Z,2600.0,김재현,201902,D시,2018-07-22,2019-07-01 11:27:00,201807
4,2019-04-22 3:09,상품A,100.0,김강현,201904,D시,2017-06-07,2019-03-01 16:57:00,201706
...,...,...,...,...,...,...,...,...,...
2994,2019-02-15 2:56,상품Y,2500.0,김정민,201902,B시,2017-07-01,2019-02-20 06:30:00,201707
2995,2019-06-22 4:03,상품M,1300.0,김재원,201906,E시,2018-03-31,2019-06-21 01:54:00,201803
2996,2019-03-29 11:14,상품Q,1700.0,김지율,201903,B시,2017-03-15,2019-07-24 22:05:00,201703
2997,2019-07-14 12:56,상품H,800.0,김승주,201907,E시,2018-07-15,2019-04-18 06:11:00,201807


In [83]:
#파일로 저장(덤프파일)
join.to_csv('20411.csv', index=False)

In [86]:
join.pivot_table(index='month', columns='item_name', values='item_price', aggfunc='sum', fill_value=0)

item_name,상품A,상품B,상품C,상품D,상품E,상품F,상품G,상품H,상품I,상품J,...,상품Q,상품R,상품S,상품T,상품U,상품V,상품W,상품X,상품Y,상품Z
month,,,,,,,,,,,,,,,,,,,,,
201901,1800,2600,5700,6800,9000,9000,7700,12800,16200,17000,...,28900,37800,38000,34000,14700,48400,29900,33600,25000,0
201902,1900,2800,7800,8400,8000,8400,9800,13600,10800,14000,...,37400,39600,41800,46000,39900,48400,55200,38400,27500,2600
201903,1700,4200,6000,6800,4500,16200,9800,14400,10800,16000,...,39100,28800,38000,24000,48300,39600,36800,50400,40000,0
201904,1700,3800,7200,8000,9000,10200,9800,8800,16200,13000,...,34000,36000,30400,32000,23100,33000,32200,38400,50000,0
201905,2400,2800,4800,5600,9500,10800,16100,12000,14400,11000,...,22100,39600,34200,32000,33600,19800,48300,38400,50000,0
201906,2400,2400,3300,7600,6500,10800,10500,10400,17100,22000,...,25500,28800,39900,24000,37800,44000,39100,36000,32500,0
201907,2000,4000,5100,6800,6000,10200,13300,15200,17100,23000,...,25500,34200,43700,42000,27300,61600,36800,43200,30000,0


In [88]:
join.pivot_table(index='month', columns='customer_name', values='item_price', aggfunc='sum', fill_value=0)

customer_name,김가온,김강민,김강현,김건우,김건희,김경민,김규민,김규현,김다온,김대현,...,김현수,김현승,김현우,김현준,김현진,김호준,정도형,정영훈,정우석,정준기
month,,,,,,,,,,,,,,,,,,,,,
201901,1000,1500,100,3700,7000,5200,1100,2200,1600,6400,...,1300,2200,1800,3500,1400,1600,5800,6600,6400,5400
201902,2500,1200,5500,2900,5700,8400,2300,5200,400,7200,...,6400,1700,7700,0,0,2200,1400,0,2500,3400
201903,6200,0,2400,500,1800,900,2300,1200,200,7000,...,3900,2600,1600,2900,2400,3700,100,600,7300,4300
201904,100,0,2100,0,1700,300,2100,9700,1800,5700,...,0,4900,2400,100,1200,2900,5000,6200,6600,800
201905,0,0,2600,2100,2400,4800,8500,5400,5400,2300,...,4300,1100,100,3700,5300,0,0,0,2400,3100
201906,6700,0,2000,0,2300,1400,8800,1500,2100,2000,...,2200,5600,2300,0,6500,3100,2700,0,0,2600
201907,5800,1900,1500,2900,900,2000,7200,800,3400,4200,...,500,1600,0,4000,0,3700,10400,2400,1200,3500


In [89]:
# 구매 이력 없는 고객 정보
away = pd.merge(sales, customer, left_on='customer_name', right_on='고객이름',how='right')
away[away['purchase_date_x'].isnull()][['customer_name','등록일']]

,customer_name,등록일
2999,NaN,2019-04-23
